<a href="https://colab.research.google.com/github/burakemretetik/makemore/blob/main/makemore_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MLP

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt", "r").read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
# build the vocabulary of characters
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [40]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one
X, Y = [], []

for w in words[:5]:

    print('\n', w)
    context = [0] * block_size
    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


 emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .

 olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .

 ava
... ---> a
..a ---> v
.av ---> a
ava ---> .

 isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .

 sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [41]:
X.shape, X.dtype, Y.shape, Y.dtype # Dataset

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [42]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [43]:
sum(p.nelement() for p in parameters) # Numbers of parameters

3481

In [45]:
for p in parameters:
    p.requires_grad = True

In [47]:
for _ in range(10):
  # Forward pass
  emb = C[X] #(32, 3, 2)
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
  logits = h @ W2 + b2 # h=(32,100) . W=(100,27)
  counts = logits.exp()
  prob = counts / counts.sum(1, keepdims=True)
  loss = -prob[torch.arange(32), Y].log().mean()
  print(loss.item())

  # Backward pass
  for p in parameters:
      p.grad = None
  loss.backward()

  # Update
  for p in parameters:
      p.data += -0.1 * p.grad

3.985849618911743
3.6028316020965576
3.262141704559326
2.961381435394287
2.6982972621917725
2.4697132110595703
2.271660327911377
2.101283550262451
1.9571771621704102
1.8374860286712646


Note: we're overfitting the model here (too few examples for a large dataset will result poor in the actual data)

### Rebuild

In [52]:
# Rebuild the dataset (with full data)

block_size = 3 # context length: how many characters do we take to predict the next one
X, Y = [], []

for w in words:
    context = [0] * block_size
    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [53]:
X.shape, X.dtype, Y.shape, Y.dtype # Dataset

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [54]:
# Setup parameters
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [55]:
for p in parameters:
    p.requires_grad = True

In [56]:
for _ in range(10):
  # Forward pass
  emb = C[X] #(32, 3, 2)
  h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
  logits = h @ W2 + b2 # h=(32,100) . W=(100,27)
  loss = F.cross_entropy(logits, Y)
  print(loss.item())

  # Backward pass
  for p in parameters:
      p.grad = None
  loss.backward()

  # Update
  for p in parameters:
      p.data += -0.1 * p.grad

19.505226135253906
17.08449363708496
15.776531219482422
14.833340644836426
14.002603530883789
13.253260612487793
12.57991886138916
11.983101844787598
11.47049331665039
11.051856994628906
